In [ ]:
import pandas as pd
import numpy as np

In [ ]:
set1 = pd.read_csv('lv2-2309.csv')

In [ ]:
#### Q1

In [ ]:
df1 = set1.copy()

job_list = df1['JOB_CODE'].unique().tolist()
city_list = df1['CITY_CATEGORY'].unique().tolist()
len(job_list) * len(city_list)

In [ ]:
from scipy import stats

max_cor = 0
max_job = ''
max_city = ''
for job in job_list:
    temp = df1[df1['JOB_CODE'] == job]
    for city in city_list:
        temp_c = temp[temp['CITY_CATEGORY'] == city]
        tmp_cor, a = stats.pearsonr(temp_c['STAY_YEARS'], temp_c['PURCHASE'])
        if max_cor < abs(tmp_cor):
            max_cor = abs(tmp_cor)
            max_job = job
            max_city = city

A = max_cor
B = df1[df1['JOB_CODE'] == max_job]['PURCHASE'].mean()

ans21 = A * B

In [ ]:
print('Q1 Result: ', ans21)
print('Q1 Answer: ', round(ans21, 3))

In [ ]:
#### Q2

In [ ]:
df2 = set1.copy()

df2_1 = df2.groupby(['GENDER','JOB_CODE','CITY_CATEGORY', 'MARITAL_STATUS'])['USER_ID'].count().reset_index()
df2_2 = df2_1.sort_values(['USER_ID'], ascending=False).reset_index(drop=True)[:20]

In [ ]:
outlier_list = []
for i in range(20):
    tmp = df2[(df2['GENDER'] == df2_2.loc[i, 'GENDER']) & (df2['JOB_CODE'] == df2_2.loc[i, 'JOB_CODE']) 
        & (df2['CITY_CATEGORY'] == df2_2.loc[i, 'CITY_CATEGORY']) & (df2['MARITAL_STATUS'] == df2_2.loc[i, 'MARITAL_STATUS'])]
    q1 = tmp['PURCHASE'].quantile(0.25)
    q3 = tmp['PURCHASE'].quantile(0.75)
    iqr = q3-q1
    total_len = len(tmp)
    outlier = len(tmp[(tmp['PURCHASE'] < q1 - iqr) | (tmp['PURCHASE'] > q3 + iqr)])    
    outlier_list.append(outlier/total_len)
ans22 = max(outlier_list)

In [ ]:
print('Q2 Result: ', ans22)
print('Q2 Answer: ', round(ans22, 3))

In [ ]:
#### Q3

In [ ]:
df3 = set1.copy()

df3['VIP'] = df3['PURCHASE'].apply(lambda x : 1 if x >= 20000 else 0)

test = pd.DataFrame({'GENDER':['M'], 'USER_AGE':['36-45'], 'JOB_CODE':['G'], 
                     'CITY_CATEGORY':['B'], 'STAY_YEARS': [3], 'MARITAL_STATUS':['Yes']})
df3_1 = df3.drop(columns = ['USER_ID','PURCHASE','VIP'])
df3_1 = pd.concat([df3_1, test])
df3_1.reset_index(drop=True,inplace=True)

g_dum = pd.get_dummies(df3_1['GENDER'], drop_first=True)
a_dum = pd.get_dummies(df3_1['USER_AGE'], drop_first=True)
j_dum = pd.get_dummies(df3_1['JOB_CODE'], drop_first=True)
c_dum = pd.get_dummies(df3_1['CITY_CATEGORY'], drop_first=True)
m_dum = pd.get_dummies(df3_1['MARITAL_STATUS'], drop_first=True)
df3_2 = pd.concat([df3_1[['STAY_YEARS']],g_dum, a_dum, j_dum, c_dum,  m_dum],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='saga', C = 100000, random_state=1234, max_iter = 4)
test_x = pd.DataFrame(df3_2.iloc[-1][:]).T
train_x = df3_2.iloc[:-1]
lr.fit(train_x, df3[['VIP']])

ans23 = lr.predict_proba(test_x)[0][1]

In [ ]:
print('Q3 Result: ', ans23)
print('Q3 Answer: ', round(ans23, 4))